In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
from transformers import BertTokenizer
import pandas as pd

In [5]:
train_df = pd.read_csv('/content/drive/MyDrive/NLP_commit/Comment_guard/unsmile_train_v1.0.tsv', delimiter='\t')
valid_df = pd.read_csv('/content/drive/MyDrive/NLP_commit/Comment_guard/unsmile_valid_v1.0.tsv', delimiter='\t')

In [6]:
train_sentences = train_df['문장']
valid_sentences = valid_df['문장']